In [1]:
import pandas as pd
import numpy as np

In [2]:
baseline = (
    pd
    .read_csv("../baseline.csv", na_filter=False)
    .pivot(columns=['model', 'transformation'], index='target', values='positive_F1')
)

baseline.to_latex("../latex/baseline.tex")
display(baseline)

model                GLM                     Random Forest
transformation      None    Root 4    PaCMAP          None
target                                                    
Beach           0.609179  0.636111  0.749245      0.687719
FallFoliage     0.634873  0.688331  0.808050      0.850258
Field           0.732591  0.732510  0.835329      0.821317
Mountain        0.493888  0.512696  0.508571      0.467227
Sunset          0.772313  0.789668  0.854932      0.880478
Urban           0.538922  0.535552  0.568182      0.508197

In [3]:
tuples = [("GLM", "None"), ("GLM", "Root 4"), ("GLM", "PaCMAP (dim 50)"), ("RF", "None")]

baseline = (
    pd
    .read_csv("../baseline-multilabel.csv", na_filter=False)
    .set_index(pd.MultiIndex.from_tuples(tuples, names=["model", "transformation"]))
    .transpose()
)

baseline[["metric", "class"]] = pd.DataFrame(baseline.index.str.split(n=2).tolist(), index=baseline.index)
baseline = baseline.set_index(["metric", "class"])

baseline.to_latex("../latex/baseline-multilabel.tex")
display(baseline)

model                          GLM                                  RF
transformation                None    Root 4 PaCMAP (dim 50)      None
metric       class                                                    
accuracy     -            0.680810  0.696288        0.735606  0.605996
precision    Beach        0.801846  0.822211        0.832931  0.888508
             Sunset       0.888126  0.902961        0.950207  0.986541
             FallFoliage  0.841089  0.869614        0.860202  0.950033
             Field        0.815072  0.825753        0.847741  0.873998
             Mountain     0.796185  0.796875        0.810440  0.897321
             Urban        0.799094  0.795139        0.779719  0.897351
recall       Beach        0.651531  0.681327        0.717711  0.546890
             Sunset       0.790157  0.782458        0.851902  0.809783
             FallFoliage  0.679872  0.732179        0.814231  0.755256
             Field        0.742795  0.737992        0.782781  0.717579
             Mountain     0.589822  0.601190        0.574161  0.417252
             Urban        0.609471  0.623393        0.618694  0.407641
Hamming-Loss -            0.135498  0.129177        0.093939  0.086234
LRAP         -            0.795100  0.809870        0.858069  0.885732

In [4]:
tuples = [("RBF", "first"), ("RBF", "second"), ("RBF", "third"), ("Poly", "first"), ("Poly", "second"), ("Poly", "third")]

svm = (
    pd
    .read_csv("multi_svm.csv", na_filter=False)
    .drop(["class_weight", "C", "kernel", "degree", "gamma", "coef0"], axis="columns")
    .set_index(pd.MultiIndex.from_tuples(tuples, names=["kernel", "position"]))
    .transpose()
)

svm[["metric", "class"]] = pd.DataFrame(svm.index.str.split(n=2).tolist(), index=svm.index)
svm = svm.set_index(["metric", "class"])

svm.to_latex("../latex/svm-multilabel.tex")
display(svm)

kernel                         RBF                          Poly            \
position                     first    second     third     first    second   
metric       class                                                           
accuracy     -            0.867424  0.803939  0.799416  0.843701  0.830368   
precision    Beach        0.869962  0.876538  0.876286  0.875120  0.873709   
             Sunset       0.973077  0.984623  0.985615  0.981838  0.966333   
             FallFoliage  0.928748  0.939236  0.939651  0.925551  0.918919   
             Field        0.901310  0.900671  0.900026  0.899205  0.905106   
             Mountain     0.864843  0.870318  0.870557  0.861979  0.870192   
             Urban        0.870200  0.877178  0.881184  0.863821  0.854287   
recall       Beach        0.847816  0.762821  0.757953  0.810423  0.806149   
             Sunset       0.914855  0.903080  0.903986  0.861413  0.885266   
             FallFoliage  0.904231  0.881154  0.881154  0.886538  0.867308   
             Field        0.861071  0.817963  0.812200  0.849304  0.837056   
             Mountain     0.810597  0.684817  0.674668  0.787666  0.760246   
             Urban        0.810584  0.736771  0.732319  0.801187  0.777077   
Hamming-Loss -            0.078355  0.066147  0.065887  0.072468  0.074199   

kernel                              
position                     third  
metric       class                  
accuracy     -            0.832186  
precision    Beach        0.875242  
             Sunset       0.977416  
             FallFoliage  0.918826  
             Field        0.900362  
             Mountain     0.858507  
             Urban        0.857143  
recall       Beach        0.794160  
             Sunset       0.859300  
             FallFoliage  0.889231  
             Field        0.837776  
             Mountain     0.769906  
             Urban        0.783136  
Hamming-Loss -            0.075238

In [5]:
svm = (
    pd.concat([
        pd.read_csv("linear_svm.csv", keep_default_na=False),
        pd.read_csv("kernel_svm.csv", keep_default_na=False, na_values=[""]),
    ])
    .sort_values("positive_F1",ascending=False)
    .groupby("kernel")
    .head(3)
    .set_index("kernel")
    .sort_index()
    .drop(["true_positive", "false_positive", "true_negative", "false_negative"], axis="columns")
    .drop(["negaitve_F1", "mean_F1", "class_weight"], axis="columns")
)

cols = np.array(["C", "transformation", "penalty", "loss", "degree", "gamma", "coef0"])
def info(series):
    valid = ~series.isna()
    names = cols[valid]
    values = series[~series.isna()].to_list()
    return ", ".join([f"{name}: {value}" for name, value in zip(names, values)])

svm["info"] = svm[cols].apply(info, axis="columns")
svm = svm.drop(cols, axis="columns")
svm = (
    svm[["info", "positive_F1"]]
    .rename({"info": "Hyper-parameters", "positive_F1": "F1-score"}, axis="columns")
    .reset_index()
    .set_index(["kernel", "Hyper-parameters"])
)

svm.to_latex("../latex/SVM-single.tex")
display(svm)

F1-score
kernel      Hyper-parameters                                            
linear      C: 0.1, transformation: root 4, penalty: l1, lo...  0.629940
            C: 0.01, transformation: root 4, penalty: l2, l...  0.627193
            C: 0.01, transformation: root 4, penalty: l2, l...  0.618824
poly        C: 0.1, degree: 5.0, gamma: 0.0034013605442176,...  0.692206
            C: 1.0, degree: 3.0, gamma: 0.0034013605442176,...  0.687500
            C: 0.01, degree: 2.0, gamma: 0.0583211843519804...  0.684636
precomputed C: 0.1, gamma: 0.0034013605442176, scale: 0.1       0.512338
            C: 10.0, gamma: 1.156925355176084e-05, scale: 1.0   0.480243
            C: 1.0, gamma: 0.0001983713753468, scale: 1.0       0.478668
rbf         C: 10.0, gamma: 0.0034013605442176                  0.690852
            C: 1.0, gamma: 0.0034013605442176                   0.687166
            C: 100.0, gamma: 0.0034013605442176                 0.676236

In [7]:
import json

with open("final_svm.json", mode="r") as file:
    result = json.load(file)

final_svm = pd.DataFrame({
    "precision": [v for k, v in result.items() if k.startswith("precision")],
    "recall": [v for k, v in result.items() if k.startswith("recall")],
}, index = [k.split()[1] for k in result.keys() if k.startswith("precision")]).transpose()

final_svm.to_latex("../latex/SVM-final.tex")
print(result["accuracy -"])
print(result["Hamming-Loss -"])
display(final_svm)

0.8630952380952381
0.07670995670995671


,Beach,Sunset,FallFoliage,Field,Mountain,Urban
precision,0.866394,0.963462,0.927052,0.899326,0.861358,0.869128
recall,0.849240,0.913043,0.887692,0.850744,0.818111,0.799209
